In [23]:
import tensorflow as tf
import numpy as np

# Load 20NG

In [11]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
news_train = fetch_20newsgroups(
    data_home='../data/20newsgroups/', 
    subset='train')
news_test = fetch_20newsgroups(
    data_home='../data/20newsgroups/', 
    subset='test')

In [13]:
vectorizer = CountVectorizer(binary=True, stop_words='english')
counts_train = vectorizer.fit_transform(news_train.data)
counts_test = vectorizer.transform(news_test.data)

In [14]:
counts_train.shape

(11314, 129796)

# Create model

In [15]:
# model params
in_dims = counts_train.shape[1]
layer_1_dims = 256
layer_2_dims = 256
out_dims = 20

In [32]:
coo = counts_train.tocoo()
idxs = np.mat([coo.row, coo.col]).transpose()
X = dict()
X['counts'] = tf.SparseTensorValue(idxs, coo.data, coo.shape)

In [30]:
input_fn = tf.estimator.inputs.numpy_input_fn(x=X, 
                                              y=news_train.target,
                                              batch_size=1000,
                                              shuffle=True)

In [18]:
def neural_net(x):
    layer_1 = tf.layers.dense(inputs=x, units=layer_1_dims, activation='relu')
    layer_2 = tf.layers.dense(inputs=layer_1, units=layer_2_dims, activation='relu')
    out_layer = tf.layers.dense(inputs=layer_2, units=out_dims)
    return out_layer

In [19]:
def model_fn(features, labels, mode):
    logits = neural_net(features)
    
    pred_classes = tf.argmax(logits, axis=1)
    pred_prob = tf.nn.softmax(logits)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
    
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})
    
    return estim_specs

In [20]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp36btq5os', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d58a545c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [33]:
model.train(input_fn, steps=1000)

AttributeError: 'SparseTensor' object has no attribute 'shape'